# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [ ]:
%pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [ ]:
%pip install --upgrade plotly

### Import Statements

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [ ]:
df = pd.read_csv('mission_launches.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
df.head(15)

In [ ]:
df.isnull().sum()

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [ ]:
# Drop rows with missing values
df.dropna(inplace=True) 

In [ ]:
df.drop('Rocket_Status', axis=1,inplace=True)
df.drop('Unnamed: 0', axis=1,inplace=True)
df.drop('Unnamed: 0.1', axis=1,inplace=True)
df.drop('Detail', axis=1, inplace=True) 

In [ ]:
df.tail(12)

## Descriptive Statistics

In [ ]:
df.describe()

In [ ]:
df.info()

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [ ]:
launch_counts = df['Organisation'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))
launch_counts.plot(kind='bar')
plt.title('Number of Space Mission Launches by Organization')
plt.xlabel('Organization')
plt.ylabel('Number of Launches')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
# print(launch_counts)

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [ ]:
missions = df['Mission_Status'].value_counts()
missions

In [ ]:
successful_missions = df['Mission_Status'].value_counts()['Success']
failed_missions = df['Mission_Status'].value_counts()['Failure'] + df['Mission_Status'].value_counts()['Partial Failure'] + df['Mission_Status'].value_counts()['Prelaunch Failure']

print(f"Success: {successful_missions}")
print(f"Failure: {failed_missions}")

In [ ]:
success_rate = successful_missions / (successful_missions + failed_missions) * 100
failure_rate = failed_missions / (successful_missions + failed_missions) * 100
success_rate, failure_rate

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [ ]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Price'] = df['Price'].fillna(0)
print("\nUnique values in Price column:")
print(df['Price'].unique())

In [ ]:
# Create the histogram
plt.figure(figsize=(10, 6))
df['Price'].hist(bins=20, edgecolor='black')
plt.title('Distribution of Space Mission Prices')
plt.xlabel('Price (USD Millions)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)

# Add a vertical line for the mean price
mean_price = df['Price'].mean()
plt.axvline(mean_price, color='red', linestyle='dashed', linewidth=2, label=f'Mean: {mean_price:.2f}M')

plt.legend()
plt.tight_layout()
plt.show()

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [ ]:
# Function to get Alpha3 code
def get_alpha3(country_name):
    try:
        return countries.get(country_name).alpha3
    except:
        return None
    
# Country name corrections
country_corrections = {
    'Russia': 'Russian Federation',
    'New Mexico': 'USA',
    'Yellow Sea': 'China',
    'Shahrud Missile Test Site': 'Iran',
    'Pacific Missile Range Facility': 'USA',
    'Barents Sea': 'Russian Federation',
    'Gran Canaria': 'USA'
}

In [ ]:
df['Country'] = df['Location'].str.split(',').str[-1].str.strip()
df['Country'] = df['Country'].replace(country_corrections)

df['Country_Code'] = df['Country'].apply(get_alpha3)

In [ ]:
country_counts = df['Country_Code'].value_counts().reset_index()
country_counts.columns = ['Country_Code', 'Mission_Count']

In [ ]:
# Create the choropleth map
fig = px.choropleth(country_counts, 
                    locations="Country_Code", 
                    color="Mission_Count",
                    hover_name="Country_Code", 
                    color_continuous_scale="matter",
                    title="Number of Space Missions by Country")

fig.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [ ]:
failed_missions = df[df['Mission_Status'] != 'Success']
failure_counts = failed_missions['Country_Code'].value_counts().reset_index()
failure_counts.columns = ['Country_Code', 'Failure_Count']

In [ ]:
# Create the choropleth map
fig = px.choropleth(failure_counts, 
                    locations="Country_Code", 
                    color="Failure_Count",
                    hover_name="Country_Code", 
                    color_continuous_scale="matter",
                    title="Number of Failed Space Missions by Country")

fig.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [ ]:
sunburst_data = df.groupby(['Country_Code', 'Organisation', 'Mission_Status']).size().reset_index(name='Count')

In [ ]:
fig = px.sunburst(
    sunburst_data,
    path=['Country_Code', 'Organisation', 'Mission_Status'],
    values='Count',
    title="Space Missions by Country, Organization, and Mission Status",
    color='Mission_Status',
    color_discrete_map={'Success': 'green', 'Failure': 'red', 'Partial Failure': 'orange'}
)

fig.update_layout(
    width=800,
    height=800
)

fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [ ]:
total_amount_spent_per_org = df.groupby('Organisation')['Price'].sum().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(12, 6))
total_amount_spent_per_org.plot(kind='bar')
plt.title('Total Amount Spent on Space Missions by Organization')
plt.xlabel('Organization')
plt.ylabel('Total Amount Spent (USD Millions)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Analyse the Amount of Money Spent by Organisation per Launch

In [ ]:
# df_data.drop('Country', axis=1, inplace=True)
df.head(5)

In [ ]:
avg_price_per_org = df.groupby('Organisation')['Price'].agg(['mean', 'count']).reset_index()
avg_price_per_org.columns = ['Organisation', 'Avg_Price', 'Launch_Count']
avg_price_per_org = avg_price_per_org.sort_values('Avg_Price', ascending=False)


In [ ]:
# Create a bar chart
fig = px.bar(avg_price_per_org, 
             x='Organisation', 
             y='Avg_Price',
             text='Avg_Price',
             hover_data=['Launch_Count'],
             title='Average Money Spent per Launch by Organisation',
             labels={'Avg_Price': 'Average Price per Launch (USD Millions)', 'Organisation': 'Organization'},
             color='Avg_Price',
             color_continuous_scale='Viridis')

# Customize the layout
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_xaxes(tickangle=45)

# Show the plot
fig.show()

In [ ]:
# Calculate and print total spent by all organizations
total_spent = df['Price'].sum()
print(f"\nTotal spent by all organizations: ${total_spent:.2f} million")

In [ ]:
# Find and print the organization with the highest total spend
org_total_spend = df.groupby('Organisation')['Price'].sum().sort_values(ascending=False)
highest_spend_org = org_total_spend.index[0]
highest_spend_amount = org_total_spend.iloc[0]
print(f"\nOrganization with the highest total spend: {highest_spend_org}")
print(f"Amount spent by {highest_spend_org}: ${highest_spend_amount:.2f} million")

# Chart the Number of Launches per Year

In [ ]:
df.drop('Location', axis=1, inplace=True)   

In [ ]:
# # Function to parse the date and remove time
# def parse_date(date_string):
#     # Remove the time part (assuming it's always at the end after a space)
#     date_only = ' '.join(date_string.split()[:4])
#     return pd.to_datetime(date_only, format='%a %b %d, %Y')

# # Convert the Date column to datetime and then directly to the desired format
# df['Date'] = df['Date'].apply(parse_date)
# df['Date'] = df['Date'].dt.strftime('%m/%Y')

In [ ]:
# Function to parse the date and extract year
def parse_date(date_string):
    # Remove the time part and parse the date
    date_only = ' '.join(date_string.split()[:4])
    return pd.to_datetime(date_only, format='%a %b %d, %Y').year

# Convert the Date column to year
df['Year'] = df['Date'].apply(parse_date)

# Count launches per year
launches_per_year = df['Year'].value_counts().sort_index().reset_index()
launches_per_year.columns = ['Year', 'Launches']

In [ ]:
# Create the line chart
plt.figure(figsize=(12, 6))
plt.plot(launches_per_year['Year'], launches_per_year['Launches'], marker='o')
plt.title('Number of Rocket Launches Per Year')
plt.xlabel('Year')
plt.ylabel('Number of Launches')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

In [ ]:
# Calculate average price per year
avg_price_per_year = df.groupby('Year')['Price'].mean().reset_index()
# Create the line chart
plt.figure(figsize=(12, 6))
plt.plot(avg_price_per_year['Year'], avg_price_per_year['Price'], marker='o')
plt.title('Average Price of Rocket Launches Over Time')
plt.xlabel('Year')
plt.ylabel('Average Price (millions USD)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)

# Format y-axis to display prices in millions
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:.1f}M'))

plt.show()

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

In [ ]:
if isinstance(df['Organisation'].dtype, pd.CategoricalDtype):
    df['Organisation'] = df['Organisation'].cat.codes
elif df['Organisation'].dtype == 'object' and isinstance(df['Organisation'].iloc[0], (list, tuple)):
    df['Organisation'] = df['Organisation'].apply(lambda x: x[0])
    
# Get the top 10 organizations by total number of launches
top_10_orgs = df['Organisation'].value_counts().nlargest(10).index

# Filter the dataframe to include only the top 10 organizations
df_top_10 = df[df['Organisation'].isin(top_10_orgs)]


In [ ]:
launches_by_org_year = df_top_10.groupby(['Organisation', 'Year']).size().unstack(fill_value=0)

# Create the line graph
plt.figure(figsize=(15, 8))

for org in launches_by_org_year.index:
    plt.plot(launches_by_org_year.columns, launches_by_org_year.loc[org], label=org, marker='o')

plt.title('Number of Launches Over Time by Top 10 Organizations')
plt.xlabel('Year')
plt.ylabel('Number of Launches')
plt.legend(title='Organization', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

In [ ]:
df1 = df[df['Year'] <= 1991].copy()
df1.reset_index(drop=True, inplace=True)

In [ ]:
df1.describe()

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

In [ ]:
ussr_countries = ['USSR', 'Russia', 'Kazakhstan', 'Ukraine']\
    
def categorize_country(country):
    if country in ussr_countries:
        return 'USSR'
    elif country == 'USA':
        return 'USA'
    else:
        return 'Other'
    
df1['Launch_Group'] = df1['Country'].apply(categorize_country)
launch_counts = df1['Launch_Group'].value_counts()

In [ ]:
# Create the pie chart
plt.figure(figsize=(10, 8))
plt.pie(launch_counts.values, labels=launch_counts.index, autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title("Comparison of Launches: USSR vs USA during Cold War (up to 1991)")

# Add a circle at the center to create a donut chart effect
center_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(center_circle)

# Add annotation in the center
plt.annotate('Cold War\nSpace Race', xy=(0, 0), xytext=(0, 0), ha='center', va='center')

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')  

# Show the chart
plt.show()

## Chart the Total Number of Mission Failures Year on Year.

In [ ]:
def is_failure(status):
    return 'Failure' in status or 'failure' in status


# Create a new column for mission failure
df1['Is_Failure'] = df1['Mission_Status'].apply(is_failure)

# Group by year and count failures
failures_by_year = df1[df1['Is_Failure']].groupby('Year').size().reset_index(name='Failures')

In [ ]:
# Create the bar chart with a smaller, more standard size
plt.figure(figsize=(10, 6))  # Adjusted figure size
plt.bar(failures_by_year['Year'], failures_by_year['Failures'], color='red', alpha=0.7)

# Customize the chart
plt.title('Total Number of Mission Failures Year on Year (Cold War Period)', fontsize=14)
plt.xlabel('Year', fontsize=10)
plt.ylabel('Number of Failures', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right', fontsize=8)

# Add value labels on top of each bar
for i, v in enumerate(failures_by_year['Failures']):
    plt.text(failures_by_year['Year'][i], v, str(v), ha='center', va='bottom', fontsize=8)

plt.tight_layout()

# Show the chart
plt.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

In [ ]:
df1.head()

In [ ]:
# Group by year and calculate failure percentage
yearly_stats = df1.groupby('Year').agg({
    'Is_Failure': ['sum', 'count']
}).reset_index()

yearly_stats.columns = ['Year', 'Failures', 'Total_Launches']
yearly_stats['Failure_Percentage'] = (yearly_stats['Failures'] / yearly_stats['Total_Launches']) * 100

In [ ]:
# Create the line plot
plt.figure(figsize=(12, 6))
plt.plot(yearly_stats['Year'], yearly_stats['Failure_Percentage'], marker='o')

# Customize the chart
plt.title('Percentage of Mission Failures Over Time (Cold War Period)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Failure Percentage', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add percentage labels
for i, txt in enumerate(yearly_stats['Failure_Percentage']):
    plt.annotate(f"{txt:.1f}%", (yearly_stats['Year'][i], txt), textcoords="offset points", xytext=(0,10), ha='center')

plt.tight_layout()

# Show the chart
plt.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [ ]:
# Group by Year and Organisation, count launches, and find the organization with max launches for each year
yearly_dominant_org = df.groupby(['Year', 'Organisation']).size().reset_index(name='Launches')
yearly_dominant_org = yearly_dominant_org.loc[yearly_dominant_org.groupby('Year')['Launches'].idxmax()]
# Sort by Year
yearly_dominant_org = yearly_dominant_org.sort_values('Year')

In [ ]:
plt.figure(figsize=(20, 10))
bars = plt.bar(yearly_dominant_org['Year'], yearly_dominant_org['Launches'], color='skyblue')

# Customize the chart
plt.title('Organisation with Most Launches Year-on-Year', fontsize=16)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Launches', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Rotate x-axis labels for better readability
plt.xticks(yearly_dominant_org['Year'], rotation=90, ha='center')

# Add organization labels on top of each bar
for bar in bars:
    year = bar.get_x() + bar.get_width() / 2
    org = yearly_dominant_org.loc[yearly_dominant_org['Year'] == int(year), 'Organisation'].values[0]
    plt.text(year, bar.get_height(), org, ha='center', va='bottom', rotation=90, fontsize=8)

plt.tight_layout()

# Show the chart
plt.show()

In [ ]:
print("Dominant organizations in the 1970s:")
print(yearly_dominant_org[yearly_dominant_org['Year'].between(1970, 1979)])

In [ ]:
print("\nDominant organizations in the 1980s:")
print(yearly_dominant_org[yearly_dominant_org['Year'].between(1980, 1989)])

In [ ]:
print("\nDominant organizations in 2018, 2019, and 2020:")
print(yearly_dominant_org[yearly_dominant_org['Year'].isin([2018, 2019, 2020])])

# That's the end for now